# Structure of Decision Tree
A Decision Tree has a tree-like structure with each internal node denoting a test on an attribute, each branch representing an outcome of the test, and each terminal node (leaf) holding a class label. Here are the parts of a Decision Tree:

- Root Node: This houses the entire dataset.
- Internal Nodes: These make decisions based on conditions.
- Edges/branches: These connections implement decision rules.
- Leaves: These are terminal nodes for making predictions.

Decisions on attributes depend on how well they help to purify the data.

The tree-building process begins with the full dataset at the root node, iteratively partitioning the data based on chosen attributes. Each child node becomes a new root that can be split further. This recursive process continues until predefined stopping criteria are met.

## Stopping Criteria for Tree Building
Standard stopping criteria include:

- Maximum Tree Depth: Limiting the maximum depth of the tree.
- Minimum Node Records: No more partitioning if less than a threshold number of records.
- Node Purity: Stop if all instances at a node belong to the same class.
These criteria ensure the model is consistent, which prevents overfitting.



## Implementing a Decision Tree using GINI
A Decision Tree is a tree-like graph that models decisions and their potential consequences. It starts at a single node, called the root, which splits into branches. Each branch, in turn, splits into more branches, forming a hierarchical network. The final branches with no further splits are referred to as leaf nodes. Each split is determined by whether the data satisfies a specific condition.

For instance, if we build a Decision Tree to predict whether a patient will recover from a disease, the root could be temperature> 101F. The tree would then split into two branches - one for yes and another for no. Each branch could further split based on another attribute, such as cough present. This process of splitting continues until we conclude the leaf nodes, such as recovery probable or recovery doubtful

In [1]:
def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini


def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right


def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))  # Find unique classes in the dataset
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):    # Exclude the last column which is the class
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

## Implementing Decision Tree in Python

Starting with the Terminal node first:

In [9]:
def create_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

## Implementing the Recursive Split
The `recurse_split` is responsible for creating children nodes:

In [ ]:
def recurse_split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])

    # If right or left groups are empty, create a terminal node.
    if not left or not right:
        node['left'] = node['right'] = create_terminal(left + right)
        return
    
    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = create_terminal(left), create_terminal(right)
        return
    
    # Process the children nodes
    if len(left) <= min_size:
        node['left'] = create_terminal(left)
    else:
        node['left'] = get_split(left)
        recurse_split(node['left'], max_depth, min_size, depth + 1)

    if len(right) <= min_size:
        node['right'] = create_terminal(right)
    else:
        node['right'] = get_split(right)
        recurse_split(node['right'], max_depth, min_size, depth + 1)

The `create_terminal` function determine the most common class value in a group of rows and assign that value as the final decision for that subset of data.

Creating the actual tree building:

In [7]:
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    recurse_split(root, max_depth, min_size, 1)
    return root

## Let's Build and print a Tree

In [10]:
dataset = [
    [5, 3, 0], [6, 3, 0], [6, 4, 0], [10, 3, 1],
    [11, 4, 1], [12, 8, 0], [5, 5, 0], [12, 4, 1]
]

max_depth = 2
min_size = 1
tree = build_tree(dataset, max_depth, min_size)

# Printing Tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

print_tree(tree)

[X1 < 10.000]
 [X1 < 5.000]
  [0]
  [0]
 [X2 < 8.000]
  [1]
  [0]


The print_tree output shows a decision tree. [X1 < 10.000] checks if feature X1 is less than 10.000. Left branch ([X1 < 5.000]) and its subsequent nodes ([0] and [0]) indicate the conditions and predictions if 'Yes'. The right branch ([X2 < 8.000]) and its nodes ([1] and [0]) cover the cases for 'No'. Indentations imply tree depth.